In [0]:
from pyspark.sql import SparkSession
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score, accuracy_score
)

#### **load from delta table directly**

In [0]:
spark = SparkSession.builder.getOrCreate()
df = spark.table("patient_risk_prediction.ml.patient_readmission_30d").toPandas()

#### **Before Encoding (Raw Data Sample)**

In [0]:
print("✅ Loaded data:", df.shape)
display(df.head(5))


✅ Loaded data: (55500, 11)


patient_name,gender,age,medical_condition,hospital,insurance_provider,stay_duration_days,billing_amount,days_to_next_admit,readmitted_within_30d,risk_level
Aaron Adams,FEMALE,38,Cancer,Hart Llc,Unitedhealthcare,19,26052.10640426882,null,0,High
Aaron Aguirre,MALE,36,Diabetes,Murray-shelton,Unitedhealthcare,15,27087.560552870273,null,0,High
Aaron Anderson,FEMALE,50,Asthma,Tanner-cox,Cigna,28,39804.6586244873,null,0,High
Aaron Anderson Md,FEMALE,20,Hypertension,Ritter Llc,Unitedhealthcare,12,16846.415799072645,null,0,High
Aaron Archer,FEMALE,47,Cancer,"Montes Case And Mendez,",Medicare,7,10602.077185418817,-7.0,0,High


#### **Encode categoricals**

#### **Basic preprocessing**

In [0]:
categorical_cols = ['gender', 'medical_condition', 'hospital', 'insurance_provider', 'risk_level']
for col in categorical_cols:
    df[col] = LabelEncoder().fit_transform(df[col].astype(str))

#### **Drop nulls if any**

In [0]:
df = df.dropna(subset=['readmitted_within_30d'])

#### **Define features and target**

In [0]:
X = df[['age', 'gender', 'medical_condition', 'hospital',
        'insurance_provider', 'billing_amount', 'stay_duration_days']]
y = df['readmitted_within_30d']

#### **Display before split**

In [0]:
print("\n🔹 Sample before split:")
display(pd.concat([X.head(3), y.head(3)], axis=1))


🔹 Sample before split:


age,gender,medical_condition,hospital,insurance_provider,billing_amount,stay_duration_days,readmitted_within_30d
38,0,2,16665,4,26052.10640426882,19,0
36,1,3,26919,4,27087.560552870273,15,0
50,0,1,35638,2,39804.6586244873,28,0


#### **Train-Test Split**

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

#### **Scale numeric features**

In [0]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

#### **Train Logistic Regression**

In [0]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

LogisticRegression(max_iter=1000)

#### **Evaluate model**

In [0]:
y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

In [0]:
print("\n================ MODEL PERFORMANCE ================")
print(f"Accuracy        : {accuracy:.4f}")
print(f"ROC-AUC Score   : {roc_auc:.4f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))



================ MODEL PERFORMANCE ================
Accuracy        : 0.9899
ROC-AUC Score   : 0.5000

Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      0.99     10988
           1       0.00      0.00      0.00       112

    accuracy                           0.99     11100
   macro avg       0.49      0.50      0.50     11100
weighted avg       0.98      0.99      0.98     11100

Confusion Matrix:
 [[10988     0]
 [  112     0]]


/databricks/python/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/databricks/python/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/databricks/python/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


#### **Display predictions**

In [0]:
pred_df = X_test.copy()
pred_df['Actual'] = y_test.values
pred_df['Predicted'] = y_pred
print("\n🔹 Predicted vs Actual sample:")
display(pred_df.head(10))


🔹 Predicted vs Actual sample:


age,gender,medical_condition,hospital,insurance_provider,billing_amount,stay_duration_days,Actual,Predicted
58,1,1,9407,0,2435.050778642094,19,0,0
46,1,4,34900,0,22274.368320227113,11,0,0
28,1,3,19339,0,48922.61389964951,4,0,0
38,1,1,6183,4,33161.536012380864,27,0,0
66,1,1,20332,2,43719.26316689056,30,0,0
68,1,4,23913,2,22038.15714582714,25,0,0
41,0,1,23363,2,24378.764495034982,27,0,0
56,1,1,34457,2,8997.03238555355,1,0,0
82,0,4,33746,1,35281.59340728224,6,0,0
83,0,5,12060,0,36262.33571168767,26,0,0
